In [1]:
from operator import index
import numpy as np
from fontTools.misc.plistlib import end_key
from scipy.special import comb
from scipy.sparse.linalg import eigs
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
## Parameter assignment
w = 1        
p = 0.2      
q = 0.2    
alpha = 0.7 
r = 0.001  
gamma = 0.8  
delta = 0.08
Z = 50
betas = 10
mu = 1/Z
N = 40
tau = 0.9;
c = 0.17
M = 10;
UW = lambda w: w**(1-gamma)/(1-gamma)
combMatrix = np.nan * np.zeros((Z+1, Z+1))
for i in range(Z+1):
    for j in range(i+1):
        combMatrix[i, j] = comb(i, j)

In [3]:
## The pafoffs of the sharing mechanism
PI_DD = p*UW((1-alpha)*w) + (1-p)*UW(w)

## refund mechanism
def p1(k,u,v,m,n):
    result = 0
    result = result + combMatrix[k, u]*combMatrix[k-u, v]*combMatrix[k-u-v, m]*(p-r)**u*(1-q-r)**v*(q+r-p)**m*r**n
    return result

def q2(k,u,v,m,n):
    if k>=M:
        result = 0
        if n!=0:
            result = result + (u+v)*UW(w-tau*c-delta*w) + m*UW(w-tau*c-delta*w+alpha*w)+n*UW((1-alpha)*w-tau*c-delta*w + (k*delta*w)/n)
        else:
            result = result + (u+v)*UW(w-tau*c)+m*UW(w-tau*c+alpha*w)
    else:
        result = 0
        if n!=0:
            result = result + (u+v)*UW(w-c-delta*w) + m*UW(w-c-delta*w+alpha*w)+n*UW((1-alpha)*w-c-delta*w + (k*delta*w)/n)
        else:
            result = result + (u+v)*UW(w-c)+m*UW(w-c+alpha*w)
        
    return result


Pi_CC_refund = np.zeros((N + 1))
for k in range(N+1):
     if k!=0:
        #I收益计算
        fuzhu1 = 0
        fuzhu2 = 0
        for u in range(k+1):
            for v in range(k+1-u):
                for m in range(k+1-u-v):
                    n=k-u-v-m
                    fuzhu1 = fuzhu1 + p1(k,u,v,m,n)*q2(k,u,v,m,n)
        Pi_CC_refund[k] = fuzhu1/k


## NoN refund mechanism
def seta(k):
    if k>=0:
        result = 1
    else:
        result = 0
    return result
def PI_CG(k):
    result = (1+p-q-2*r)*UW(w-tau*c-delta*w)+(q+r-p)*UW(w-tau*c+alpha*w-delta*w)
    fuzhu1 = 0;
    for h in range(k):
        fuzhu1 = fuzhu1 + r*combMatrix[k-1,h]*r**h*(1-r)**(k-1-h)*UW((1-alpha)*w-tau*c-delta*w+(k*delta*w)/(h+1))
    return result+fuzhu1
def PI_CI(k):
    result = (1+p-q-2*r)*UW(w-c-delta*w) + (q+r-p)*UW(w-c+alpha*w-delta*w)
    fuzhu1 = 0;
    for h in range(k):
        fuzhu1 = fuzhu1 + r*combMatrix[k-1,h]*r**h*(1-r)**(k-1-h)*UW((1-alpha)*w-c-delta*w+(k*delta*w)/(h+1))
    return result+fuzhu1

Pi_CC_NoN_refund = np.zeros((N + 1))
for k in range(N+1):
    if k>=1:
       Pi_CC_NoN_refund[k] = seta(k-M)*PI_CG(k) + (1-seta(k-M))*PI_CI(k)

In [4]:
## The refund mechanism is Markov
def fc(i):
    fuzhu1 = 0
    for k in range(N):
        if k<=i-1 and N-1-k<=Z-i and N-1<=Z-1:
            fuzhu1 = fuzhu1 + Pi_CC_refund[k+1]*combMatrix[i-1,k]* combMatrix[Z-i,N-1-k]/combMatrix[Z-1,N-1]
        else:
            fuzhu1 = fuzhu1+0

    return fuzhu1

def fd(i):
    fuzhu1 = PI_DD

    return fuzhu1

import numpy as np
##转移概率计算
def TC_N(i, Fc, Fn):
    term1 = (1 - mu) * (i / Z) * ((Z - i) / (Z-1)) * (1 + np.exp(betas * (Fc - Fn))) ** -1
    term2 = mu * (i / Z)
    return term1 + term2

def TN_C(i, Fn, Fc):
    term1 = (1 - mu) * (i / Z) * ((Z - i) / (Z-1)) * (1 + np.exp(betas * (Fn - Fc))) ** -1
    term2 = mu * ((Z - i) / Z)
    return term1 + term2
    
p_shift = np.zeros((Z+1,Z+1))
for i in range(Z):
    p_shift[i,i+1] = TN_C(i,fd(i),fc(i))
    p_shift[i+1,i] = TC_N(i+1,fc(i+1),fd(i+1))

p_shift += np.diag(1 - np.sum(p_shift, axis=1))  
values, dis_tr = eigs(p_shift.T, k=1, which='LM')  
v = dis_tr[:, 0].real  
v = abs(v)
v_average = v/sum(v)
result = 0
for i in range(Z+1):
    result += v_average[i]*i
Average_insurance_participation_rate_refund = result/Z

In [5]:
## The NoN-refund mechanism is Markov
def fcNoN(i):
    fuzhu1 = 0
    for k in range(N):
        if k<=i-1 and N-1-k<=Z-i and N-1<=Z-1:
            fuzhu1 = fuzhu1 + Pi_CC_NoN_refund[k+1]*combMatrix[i-1,k]* combMatrix[Z-i,N-1-k]/combMatrix[Z-1,N-1]
        else:
            fuzhu1 = fuzhu1+0

    return fuzhu1

p_shift = np.zeros((Z+1,Z+1))
for i in range(Z):
    p_shift[i,i+1] = TN_C(i,fd(i),fcNoN(i))
    p_shift[i+1,i] = TC_N(i+1,fcNoN(i+1),fd(i+1))

p_shift += np.diag(1 - np.sum(p_shift, axis=1))  
values, dis_tr = eigs(p_shift.T, k=1, which='LM')  
v = dis_tr[:, 0].real  
v = abs(v)
v_average = v/sum(v)
result = 0
for i in range(Z+1):
    result += v_average[i]*i
Average_insurance_participation_rate_NoN_refund = result/Z

In [6]:
print('Refund mechanism average index insurance participation rate:',Average_insurance_participation_rate_refund,"\nNoN-refund mechanism average index insurance participation rate:",Average_insurance_participation_rate_NoN_refund)

Refund mechanism average index insurance participation rate: 0.9202589298201973 
NoN-refund mechanism average index insurance participation rate: 0.06276006800127265
